# INM432 Big Data Coursework 2016/207 Part 2: Spakr Pipelines and Evaluation of Scaling of Algorithms

### Team Members: Ryan Nazareth and Aimore Dutra 

## a) Choice of Dataset and Task 


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
# the imports are used creating the data frame

spark = SparkSession.builder.getOrCreate() # create a SparkSession 
# this gets us an RDD. (could also be done with RDD.textFile in this case)
lines = spark.read.text("hdfs://saltdean/data/movielens/sample_movielens_ratings.txt").rdd 
# now split the lines at the '::'
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
ratings.createOrReplaceTempView('ratings') # register the DataFrame so that we can use it with Spark SQL.
(training, test) = ratings.randomSplit([0.8, 0.2]) # split into test and training set
print(training.describe()) # just for testing, should show the four columns
print(training.count()) # just fore testing, should be around 1188

DataFrame[summary: string, movieId: string, rating: string, timestamp: string, userId: string]
1223


## b) Machine Learning Pipeline 

Now create an ALS estimator and a parameter grid to explore different values for the `rank` and `regParam` parameter of the ALS. 

### Choice of processing steps:

### Learning algorithms: 

### Parameter settings:


In [2]:
SQL1 = 'SELECT AVG(rating) FROM ratings'
row = spark.sql(SQL1).collect()[0] # get the single row with the result

meanRating = row['avg(rating)'] # access Row as a map 
print('meanRating',meanRating)

se_rdd = test.rdd.map(lambda row: Row(se = pow(row['rating']-meanRating,2)) ) 
se_df = spark.createDataFrame(se_rdd) 
se_df.createOrReplaceTempView('se')
print('se_df',se_df)
SQL2 = 'SELECT AVG(se) FROM se'
row = spark.sql(SQL2).collect()[0]
meanSE = row['avg(se)'] # access Row as a map 
print('meanSE',meanSE)

meanRating 1.7741505662891406
se_df DataFrame[se: double]
meanSE 1.3087042746844497


## c) Evaluating Performance of Pipeline using training and test sets


In [3]:
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, rank=5, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating")

regEval = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

crossVal = CrossValidator(estimator=als, estimatorParamMaps=paramGrid, evaluator=regEval)
print('starting cross-validation')
cvModel = crossVal.fit(training)
print('finished cross-validation')

starting cross-validation
finished cross-validation


## d) Implementing a parameter grid 

Implementing a parameter grid (using pyspark.ml.tuning.ParamGridBuilder[source]), varying at least one feature preprocessing step, one machine learning parameter, and the training set size. 

In [10]:
from pyspark.ml.tuning import ParamGridBuilder


paramGrid = ParamGridBuilder() \
    .addGrid(als.regParam, [0.03,0.1,0.3]) \
    .addGrid(als.rank, [5,10,50]).build()
    #    .addGrid(als.rank, [3,10,30,100,300]).build() 

print(cvModel.bestModel.rank)
paramMap = list(zip(cvModel.getEstimatorParamMaps(),cvModel.avgMetrics))
paramMax = max(paramMap, key=lambda x: x[1])
print(paramMax)
# Evaluate the model by computing the RMSE on the test data
predictions = cvModel.transform(test)
rmse = regEval.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

10
({Param(parent='ALS_4c32810e0999be3026e9', name='rank', doc='rank of the factorization'): 10, Param(parent='ALS_4c32810e0999be3026e9', name='regParam', doc='regularization parameter (>= 0).'): 0.03}, 5.241072647683341)
Root-mean-square error = 1.0655693202120597
